Create problem

In [1]:
	
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic

In [2]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

In [3]:
problem = create_problem(10, density=0.15, noise_level=10, negative_values=False)

In [4]:
problem

array([[    0.,    inf, 10573.,    inf,  8430.,    inf,    inf,   831.,
         1977.,    inf],
       [   inf,     0.,    inf,    inf,    inf,  2434.,    inf,    inf,
           inf,  6771.],
       [   inf,    inf,     0.,    inf,    inf,    inf,    inf,    inf,
           inf,  2208.],
       [   inf,    inf,  8563.,     0.,  7768.,    inf,    inf,    inf,
           inf,    inf],
       [ 7330.,    inf,    inf,  8367.,     0.,    inf,    inf,    inf,
           inf,    inf],
       [   inf,    inf,    inf,    inf,    inf,     0.,    inf,    inf,
           inf,    inf],
       [   inf,    inf,    inf,    inf,    inf,    inf,     0.,  5247.,
           inf,    inf],
       [   inf,    inf,  5287.,    inf,    inf,    inf,    inf,     0.,
           inf,    inf],
       [   inf,    inf,    inf, 10443.,  8363.,    inf,  5258.,    inf,
            0.,    inf],
       [   inf,    inf,    inf,    inf,    inf,    inf,  7850.,  7752.,
           inf,     0.]])

In [5]:
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [6]:
for s, d in combinations(range(problem.shape[0]), 2):
    try:
        path = nx.shortest_path(G, s, d, weight='weight')
        cost = cost = nx.path_weight(G, path, weight='weight')
    except nx.NetworkXNoPath:
        path = None
        cost = np.inf
    ic(s, d, path, cost)
None

ic| s: 0, d: 1, path: None, cost: inf
ic| s: 0, d: 2, path: [0, 7, 2], cost: 6118.0
ic| s: 0, d: 3, path: [0, 8, 3], cost: 12420.0
ic| s: 0, d: 4, path: [0, 4], cost: 8430.0
ic| s: 0, d: 5, path: None, cost: inf
ic| s: 0, d: 6, path: [0, 8, 6], cost: 7235.0
ic| s: 0, d: 7, path: [0, 7], cost: 831.0
ic| s: 0, d: 8, path: [0, 8], cost: 1977.0
ic| s: 0, d: 9, path: [0, 7, 2, 9], cost: 8326.0
ic| s: 1, d: 2, path: [1, 9, 7, 2], cost: 19810.0
ic| s: 1, d: 3, path: None, cost: inf
ic| s: 1, d: 4, path: None, cost: inf
ic| s: 1, d: 5, path: [1, 5], cost: 2434.0
ic| s: 1, d: 6, path: [1, 9, 6], cost: 14621.0
ic| s: 1, d: 7, path: [1, 9, 7], cost: 14523.0
ic| s: 1, d: 8, path: None, cost: inf
ic| s: 1, d: 9, path: [1, 9], cost: 6771.0
ic| s: 2, d: 3, path: None, cost: inf
ic| s: 2, d: 4, path: None, cost: inf
ic| s: 2, d: 5, path: None, cost: inf
ic| s: 2, d: 6, path: [2, 9, 6], cost: 10058.0
ic| s: 2, d: 7, path: [2, 9, 7], cost: 9960.0
ic| s: 2, d: 8, path: None, cost: inf
ic| s: 2, d: 9, pat

In [7]:
def solve_instance_dijkstra(
        size: int,
        density: float,
        noise_level: float,
        negative_values: bool,
        seed: int = 42,
):
    problem = create_problem(
        size,
        density=density,
        noise_level=noise_level,
        negative_values=negative_values,
        seed=seed,
    )

    if (problem < 0).any():
        print("Error: Negative values in the problem matrix.")
        return problem, []   

    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

    results = []   
    for s, d in combinations(range(problem.shape[0]), 2):
        try:
            path = nx.shortest_path(G, source=s, target=d, weight="weight")
            cost = nx.path_weight(G, path, weight="weight")
        except nx.NetworkXNoPath:
            path = None
            cost = np.inf

        #ic("[DJ]", size, density, noise_level, "s=", s, "d=", d, "cost=", cost, "path=", path)

        results.append((s, d, path, cost))

    return problem, results


In [8]:
def solve_instance_bellman_ford(
        size: int,
        density: float,
        noise_level: float,
        negative_values: bool,
        seed: int = 42,
):
    problem = create_problem(
        size,
        density=density,
        noise_level=noise_level,
        negative_values=negative_values,
        seed=seed
    )

    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

    results = []  # [(s, d, path, cost, status), ...]

    for s, d in combinations(range(problem.shape[0]), 2):
        try:
            path = nx.bellman_ford_path(G, source=s, target=d, weight="weight")
            cost = nx.path_weight(G, path, weight="weight")
            status = "ok"               # path found, no negative cycle

        except nx.NetworkXNoPath:
            path = None
            cost = np.inf
            status = "no_path"          # no path found

        except nx.NetworkXUnbounded:
            path = None
            cost = -np.inf
            status = "neg_cycle"        # negative cycle detected

        # cost check
        if status == "ok" and cost <= 0:
            status = "not_positive"     # total cost <= 0

        
        #ic("[BF]", size, density, noise_level, "s=", s, "d=", d,
        # "status=", status, "cost=", cost, "path=", path)

        results.append((s, d, path, cost, status))

    return problem, results


In [9]:
import numpy as np
import networkx as nx
from itertools import product

# get a copy of the coordinates used in create_problem
def get_coords(size: int, seed: int = 42) -> np.ndarray:
    rng = np.random.default_rng(seed)
    coords = rng.random(size=(size, 2))  # same as in create_problem
    return coords

In [10]:
def solve_instance_astar(
        size: int,
        density: float,
        noise_level: float,
        negative_values: bool,
        seed: int = 42,
):
    problem = create_problem(
        size,
        density=density,
        noise_level=noise_level,
        negative_values=negative_values,
        seed=seed,
    )

    if (problem < 0).any():
        print("Error: negative edge found, A* not appropriate for this instance.")
        return problem, []

    coords = get_coords(size, seed=seed)

    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

    def heuristic(u: int, v: int) -> float:
        dx = coords[u, 0] - coords[v, 0]
        dy = coords[u, 1] - coords[v, 1]
        return np.sqrt(dx * dx + dy * dy) * 1000

    results = []   # [(s, d, path, cost), ...]
    for s, d in combinations(range(problem.shape[0]), 2):
        try:
            path = nx.astar_path(
                G,
                source=s,
                target=d,
                heuristic=heuristic,
                weight="weight",
            )
            cost = nx.path_weight(G, path, weight="weight")
        except nx.NetworkXNoPath:
            path = None
            cost = np.inf

        #ic("[A*]", size, density, noise_level, "s=", s, "d=", d, "cost=", cost, "path=", path)

        results.append((s, d, path, cost))

    return problem, results


#start coding


parameter

size = [10, 20, 50, 100, 200, 500, 1000]
density = [0.2, 0.5, 0.8, 1]
noise_level = [0.0, 0.1, 0.5, 0.8]
negative_values = [F, T]


In [13]:


sizes = [10, 50, 100]
densities = [0.2, 0.8]
noise_levels = [0.0, 0.5]
negative_values_list = [False, True]

results_dj = []
results_astar = []
results_bf = []

for sz in sizes:
    for dn in densities:
        for nl in noise_levels:
            for nv in negative_values_list:
                print(f"\n=== setting: size={sz}, density={dn}, noise={nl}, negative={nv} ===")

                if not nv:
                    # ---------- Dijkstra ----------
                    problem_dj, res_dj = solve_instance_dijkstra(
                        size=sz,
                        density=dn,
                        noise_level=nl,
                        negative_values=nv,
                        seed=42,
                    )

                    num_pairs = len(res_dj)
                    finite_costs = [
                        cost for (_, _, path, cost) in res_dj
                        if path is not None and np.isfinite(cost)
                    ]
                    num_reachable = len(finite_costs)
                    avg_cost = np.mean(finite_costs) if finite_costs else np.inf

                    
                    if res_dj:
                        s0, d0, path0, cost0 = res_dj[0]
                        print(f"[DJ] pairs={num_pairs}, reachable={num_reachable}, "
                           f"avg_cost={float(avg_cost):.1f}, sample: {s0}->{d0}, cost={cost0}, path={path0}")
                    else:
                        print(f"[DJ] no pairs?")

                    results_dj.append({
                        "size": sz,
                        "density": dn,
                        "noise": nl,
                        "negative": nv,
                        "num_pairs": num_pairs,
                        "num_reachable": num_reachable,
                        "avg_cost": avg_cost,
                    })

                    # ---------- A* ----------
                    problem_astar, res_astar = solve_instance_astar(
                        size=sz,
                        density=dn,
                        noise_level=nl,
                        negative_values=nv,
                        seed=42,
                    )

                    num_pairs_astar = len(res_astar)
                    finite_costs_astar = [
                        cost for (_, _, path, cost) in res_astar
                        if path is not None and np.isfinite(cost)
                    ]
                    num_reachable_astar = len(finite_costs_astar)
                    avg_cost_astar = np.mean(finite_costs_astar) if finite_costs_astar else np.inf

                    if res_astar:
                        s0, d0, path0, cost0 = res_astar[0]
                        print(f"[A*] pairs={num_pairs_astar}, reachable={num_reachable_astar}, "
                           f"avg_cost={float(avg_cost_astar):.1f}, sample: {s0}->{d0}, cost={cost0}, path={path0}")
                    else:
                        print(f"[A*] no pairs?")

                    results_astar.append({
                        "size": sz,
                        "density": dn,
                        "noise": nl,
                        "negative": nv,
                        "num_pairs": num_pairs_astar,
                        "num_reachable": num_reachable_astar,
                        "avg_cost": avg_cost_astar,
                    })

                else:
                    # ---------- Bellman-Ford ----------
                    problem_bf, res_bf = solve_instance_bellman_ford(
                        size=sz,
                        density=dn,
                        noise_level=nl,
                        negative_values=nv,
                        seed=42,
                    )

                    num_pairs_bf = len(res_bf)
                    status_counts = {"ok": 0, "no_path": 0, "neg_cycle": 0, "not_positive": 0}
                    finite_costs_bf = []

                    for (_, _, path, cost, status) in res_bf:
                        status_counts[status] = status_counts.get(status, 0) + 1
                        if status == "ok" and np.isfinite(cost):
                            finite_costs_bf.append(cost)

                    avg_cost_bf = np.mean(finite_costs_bf) if finite_costs_bf else np.inf

                    print(f"[BF] pairs={num_pairs_bf}, "
                       f"ok={status_counts.get('ok',0)}, "
                       f"no_path={status_counts.get('no_path',0)}, "
                       f"neg_cycle={status_counts.get('neg_cycle',0)}, "
                       f"not_positive={status_counts.get('not_positive',0)}, "
                       f"avg_cost(ok)={float(avg_cost_bf):.1f}")

                    results_bf.append({
                        "size": sz,
                        "density": dn,
                        "noise": nl,
                        "negative": nv,
                        "num_pairs": num_pairs_bf,
                        "status_counts": status_counts,
                        "avg_cost_ok": avg_cost_bf,
                    })

print(f"\n final summary: dj_runs={len(results_dj)}, "
   f"astar_runs={len(results_astar)}, bf_runs={len(results_bf)}")



=== setting: size=10, density=0.2, noise=0.0, negative=False ===
[DJ] pairs=45, reachable=44, avg_cost=1319.5, sample: 0->1, cost=inf, path=None
[A*] pairs=45, reachable=44, avg_cost=1319.5, sample: 0->1, cost=inf, path=None

=== setting: size=10, density=0.2, noise=0.0, negative=True ===
[BF] pairs=45, ok=44, no_path=1, neg_cycle=0, not_positive=0, avg_cost(ok)=1319.5

=== setting: size=10, density=0.2, noise=0.5, negative=False ===
[DJ] pairs=45, reachable=44, avg_cost=1814.8, sample: 0->1, cost=inf, path=None
[A*] pairs=45, reachable=44, avg_cost=1814.8, sample: 0->1, cost=inf, path=None

=== setting: size=10, density=0.2, noise=0.5, negative=True ===
[BF] pairs=45, ok=41, no_path=1, neg_cycle=0, not_positive=3, avg_cost(ok)=1114.5

=== setting: size=10, density=0.8, noise=0.0, negative=False ===
[DJ] pairs=45, reachable=45, avg_cost=557.0, sample: 0->1, cost=272.0, path=[0, 1]
[A*] pairs=45, reachable=45, avg_cost=557.0, sample: 0->1, cost=272.0, path=[0, 1]

=== setting: size=10,

In [ ]:
import heapq
import math

def dijkstra(n, adj, source):
    dist = [math.inf] * n
    prev = [-1] * n
    dist[source] = 0.0

    pq = [(0.0, source)]

    while pq:
        cur_dist, u = heapq.heappop(pq)

        if cur_dist > dist[u]:
            continue


        for v, w in adj[u]:
            if w < 0:
                raise ValueError("Dijkstra only for positive value,  w = {}".format(w))

            new_dist = dist[u] + w
            if new_dist < dist[v]:
                dist[v] = new_dist
                prev[v] = u
                heapq.heappush(pq, (new_dist, v))

    return dist, prev


def reconstruct_path(prev, s, t):
    path = []
    cur = t
    while cur != -1:
        path.append(cur)
        if cur == s:
            break
        cur = prev[cur]

    if not path or path[-1] != s:
        return None
    path.reverse()
    return path


In [ ]:
import numpy as np

def matrix_to_adj(cost_matrix: np.ndarray):
    
    n = cost_matrix.shape[0]
    adj = [[] for _ in range(n)]
    for u in range(n):
        for v in range(n):
            w = cost_matrix[u, v]
            if not np.isinf(w):         
                adj[u].append((v, float(w)))
    return adj


In [22]:
problem = create_problem(
    size=10,
    density=0.5,
    noise_level=0.0,
    negative_values=False,  
    seed=42,
)

adj = matrix_to_adj(problem)

s, t = 0, 9
dist, prev = dijkstra(n=problem.shape[0], adj=adj, source=s)
path = reconstruct_path(prev, s, t)
cost = dist[t]

print("problem:\n", problem)
print("path:", path)
print("cost:", cost)


problem:
 [[   0.   inf  866.  347.  646.  633.  405.  392.  435.   inf]
 [ 272.    0.   inf  132.   inf  539.   inf  627.   inf   73.]
 [ 866.  813.    0.   inf  526.  281.  571.   inf 1021.  810.]
 [  inf   inf  693.    0.  717.   inf  123.   inf   inf   inf]
 [ 646.   inf  526.  717.    0.  535.   inf   inf  576.   inf]
 [ 633.   inf  281.   inf  535.    0.  292.   inf   inf   inf]
 [  inf   inf   inf  123.   inf   inf    0.  628.   inf  265.]
 [  inf  627.  826.   inf  386.  703.   inf    0.  198.  558.]
 [  inf   inf   inf  751.  576.  882.  764.   inf    0.   inf]
 [  inf   inf  810.   inf   inf   inf  265.  558.   inf    0.]]
path: [0, 6, 9]
cost: 670.0


In [18]:
def all_pairs_dijkstra_with_handwritten(problem):
    n = problem.shape[0]
    adj = matrix_to_adj(problem)
    results = []
    for s in range(n):
        dist, prev = dijkstra(n, adj, source=s)
        for t in range(s + 1, n):
            path = reconstruct_path(prev, s, t) if math.isfinite(dist[t]) else None
            cost = dist[t]
            results.append((s, t, path, cost))
    return results


In [ ]:
size = 10
problem = create_problem(
    size,
    density=0.5,
    noise_level=0.0,
    negative_values=False,   #must be non-negative for Dijkstra
    seed=42,
)

results = all_pairs_dijkstra_with_handwritten(problem)

for item in results:
    s, t, path, cost = item
    print(f"s={s}, t={t}, cost={cost}, path={path}")


s=0, t=1, cost=1019.0, path=[0, 7, 1]
s=0, t=2, cost=866.0, path=[0, 2]
s=0, t=3, cost=347.0, path=[0, 3]
s=0, t=4, cost=646.0, path=[0, 4]
s=0, t=5, cost=633.0, path=[0, 5]
s=0, t=6, cost=405.0, path=[0, 6]
s=0, t=7, cost=392.0, path=[0, 7]
s=0, t=8, cost=435.0, path=[0, 8]
s=0, t=9, cost=670.0, path=[0, 6, 9]
s=1, t=2, cost=820.0, path=[1, 5, 2]
s=1, t=3, cost=132.0, path=[1, 3]
s=1, t=4, cost=849.0, path=[1, 3, 4]
s=1, t=5, cost=539.0, path=[1, 5]
s=1, t=6, cost=255.0, path=[1, 3, 6]
s=1, t=7, cost=627.0, path=[1, 7]
s=1, t=8, cost=707.0, path=[1, 0, 8]
s=1, t=9, cost=73.0, path=[1, 9]
s=2, t=3, cost=694.0, path=[2, 6, 3]
s=2, t=4, cost=526.0, path=[2, 4]
s=2, t=5, cost=281.0, path=[2, 5]
s=2, t=6, cost=571.0, path=[2, 6]
s=2, t=7, cost=1199.0, path=[2, 6, 7]
s=2, t=8, cost=1021.0, path=[2, 8]
s=2, t=9, cost=810.0, path=[2, 9]
s=3, t=4, cost=717.0, path=[3, 4]
s=3, t=5, cost=974.0, path=[3, 2, 5]
s=3, t=6, cost=123.0, path=[3, 6]
s=3, t=7, cost=751.0, path=[3, 6, 7]
s=3, t=8, cost=9

In [26]:
# compare with networkx

(s, t, my_path, my_cost) = results[0]

masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

nx_path = nx.shortest_path(G, source=s, target=t, weight="weight")
nx_cost = nx.path_weight(G, nx_path, weight="weight")

print("my   :", my_path, my_cost)
print("nx   :", nx_path, nx_cost)


my   : [0, 7, 1] 1019.0
nx   : [0, 7, 1] 1019.0
